# A (Possible) Code for Calibrating the Tagger Timing

First, we import the tools we'll need:

In [1]:
import Acqu as aq
import AcquDetector as aqdet
import ROOT
import numpy as np
from rootpy.plotting import histogram, Hist2D, Hist, Canvas

Welcome to JupyROOT 6.16/00


Import the data to be analyzed:

In [2]:
inFile = '/scratch/2019-05_Timepix/Timepix_33.dat'
aq.openFile(inFile)

Mk2 Data


Load in the detector file for the Tagger:

In [3]:
aqdet.LoadDetectors(['taggerNewer.json'])

taggerNewer.json


Now we plot the data for the tagger channels and apply a gaussian fit (with an offset) to them (note that onl tbe first 10 channels are done, as it's very slow running to do all the channels at once):

In [18]:
taggerChannels = aqdet.Channels['Tagger']                 # number of channels in the tagger (368)

myFit = ROOT.TF1("myFit","gaus(0)+pol0(3)")               # make a gaussian fit with an offset/background
myFit.SetParNames("constant","mean","stdev","offset")

histos = [None]*20          # set up an array of 10 histograms to fill
offset = [None]*20
bg = [None]*20

for i in range(20):                     # for each tagger channel
    histos[i] = Hist(1300,-500,800)     # set up a histogram for that tagger channel

def plotCalTagger():
    data = aqdet.Calibrate(aq.adcArray)
    taggerTimes = aqdet.Arrays['Tagger']['Time']
    
    for dat in taggerTimes:
        for time in dat['Time']:
            for i in range(20):
                if (dat['channel']==i):
                    histos[i].Fill(time)
                    
    if(aq.eventNo%5000==0):
        print("Events Processed: ",aq.eventNo)
    
aq.runFunction(plotCalTagger,0,500000)

ROOT.enableJSVis()

c1 = ROOT.TCanvas("my1","Title",1000,5000)
c1.Divide(2,10)

for i in range(20):
    bg[i] = histos[i].GetBinContent(500)
    print(bg[i])
    myFit.SetParameter(1,140)                                 # look for a peak near 150
    myFit.SetParameter(2,1.1)   # look for a peak with a stdev of about 1
    myFit.SetParameter(3,bg[i])    
    c1.cd(i+1)
    histos[i].SetTitle("Channel "+str(i))
    histos[i].Draw("colz")
    histos[i].Fit('myFit','','',-300,600)
    offset[i] = histos[i].Fit('myFit','SQ').Get().Parameter(3)
c1.Draw()

('Events Processed: ', 5000)
('Events Processed: ', 10000)
('Events Processed: ', 15000)
('Events Processed: ', 20000)
('Events Processed: ', 25000)
('Events Processed: ', 30000)
('Events Processed: ', 35000)
('Events Processed: ', 40000)
('Events Processed: ', 45000)
('Events Processed: ', 50000)
('Events Processed: ', 55000)
('Events Processed: ', 60000)
('Events Processed: ', 65000)
('Events Processed: ', 70000)
('Events Processed: ', 75000)
('Events Processed: ', 80000)
('Events Processed: ', 85000)
('Events Processed: ', 90000)
('Events Processed: ', 95000)
('Events Processed: ', 100000)
('Events Processed: ', 105000)
('Events Processed: ', 110000)
('Events Processed: ', 115000)
('Events Processed: ', 120000)
('Events Processed: ', 125000)
('Events Processed: ', 130000)
('Events Processed: ', 135000)
('Events Processed: ', 140000)
('Events Processed: ', 145000)
('Events Processed: ', 150000)
('Events Processed: ', 155000)
('Events Processed: ', 160000)
('Events Processed: ', 16500

('Events Processed: ', 500000)
82.0
98.0
77.0
103.0
85.0
63.0
68.0
75.0
59.0
54.0
54.0
56.0
49.0
42.0
52.0
28.0
51.0
35.0
53.0
45.0
35.0
43.0
39.0
34.0
32.0
40.0
21.0
28.0
21.0
31.0
31.0
37.0
23.0
34.0
18.0
27.0
14.0
19.0
35.0
13.0
26.0
21.0
21.0
14.0
26.0
21.0
18.0
13.0
13.0
17.0
19.0
16.0
15.0
12.0
11.0
15.0
12.0
15.0
18.0
16.0
11.0
13.0
14.0
17.0
9.0
15.0
17.0
9.0
18.0
10.0
14.0
16.0
10.0
9.0
11.0
11.0
8.0
8.0
8.0
8.0
5.0
11.0
6.0
7.0
10.0
8.0
9.0
8.0
13.0
8.0
9.0
7.0
9.0
12.0
9.0
9.0
12.0
4.0
7.0
9.0
 FCN=998.687 FROM MIGRAD    STATUS=CONVERGED     274 CALLS         275 TOTAL
                     EDM=9.08873e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  constant     1.15731e+02   1.22967e+01   1.52173e-01  -7.65392e-05
   2  mean         1.41356e+02   1.15690e-01   1.78588e-03   5.04482e-03
   3  stdev        1.05835e+00   1.10132e-0

Setting peaks to 800?:

In [30]:
bins = 1200
ROOT.enableJSVis()
c2 = ROOT.TCanvas("my2","Title",1000,2500)
c2.Divide(2,5)

histos2 = [None]*10
diff = [None]*10
for i in range(10):
    histos2[i] = Hist(bins,300,1500)
    diff[i] = int(round(means[i]-800))

for i in range(10):
    for j in range(bins):
        histos2[i].SetBinContent(j-diff[i],histos2[i].GetBinContent(j))
                 
    c2.cd(i+1)
    histos2[i].SetTitle("Channel "+str(i))
    histos2[i].Draw("colz")
    myFit.SetParameter(1,800)
    myFit.SetParameter(2,5)
    histos2[i].Fit('myFit')
    
c2.Draw()

 FCN=2116.43 FROM MIGRAD    STATUS=CONVERGED     131 CALLS         132 TOTAL
                     EDM=1.60888e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  constant     4.44745e+01   5.87332e-01   1.14504e-02   9.65271e-04
   2  mean         8.02404e+02   1.97847e-01   3.94582e-03  -1.40860e-03
   3  stdev        8.94571e+00   1.59068e-01   2.86422e-03   2.43431e-04
   4  offset       2.00018e+00   7.12817e-02   1.56790e-03  -9.93754e-04
 FCN=1761.82 FROM MIGRAD    STATUS=CONVERGED     108 CALLS         109 TOTAL
                     EDM=1.36912e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  constant     3.56476e+01   6.36852e-01   1.16041e-02   2.90993e-04
   2  mean         7.99100e+02   

Testing Zone:

In [17]:
taggerChannels = aqdet.Channels['Tagger'] # number of channels in the tagger (368)

bins = 1300
h1 = Hist(bins,-500,800)

def plotCalTagger():
    data = aqdet.Calibrate(aq.adcArray)
    taggerTimes = aqdet.Arrays['Tagger']['Time']
    
    for dat in taggerTimes:
        for time in dat['Time']:
            if (dat['channel']==150):
                h1.Fill(time)
                
    if(aq.eventNo%10000==0):
        print("Events Processed: ",aq.eventNo)
        
    
aq.runFunction(plotCalTagger,0,500000)

ROOT.enableJSVis()

c1 = ROOT.TCanvas("my1","Title",1000,1000)

h1.SetTitle("Channel 20")
h1.GetXaxis().SetTitle("Time")
h1.Draw("colz")
myFit = ROOT.TF1("myFit","gaus(0)+pol0(3)")
myFit.SetParameter(1,150)
myFit.SetParameter(2,10)
h1.Fit('myFit','','',-200,600)


con = h1.Fit('myFit', 'SQ').Get().Parameter(0)
mean = h1.Fit('myFit', 'SQ').Get().Parameter(1)
stdev = h1.Fit('myFit','SQ').Get().Parameter(2)
offset = h1.Fit('myFit','SQ').Get().Parameter(3)
print(con,mean,stdev,offset)

c1.Draw()

('Events Processed: ', 10000)
('Events Processed: ', 20000)
('Events Processed: ', 30000)
('Events Processed: ', 40000)
('Events Processed: ', 50000)
('Events Processed: ', 60000)
('Events Processed: ', 70000)
('Events Processed: ', 80000)
('Events Processed: ', 90000)
('Events Processed: ', 100000)
('Events Processed: ', 110000)
('Events Processed: ', 120000)
('Events Processed: ', 130000)
('Events Processed: ', 140000)
('Events Processed: ', 150000)
('Events Processed: ', 160000)
('Events Processed: ', 170000)
('Events Processed: ', 180000)
('Events Processed: ', 190000)
('Events Processed: ', 200000)
('Events Processed: ', 210000)
('Events Processed: ', 220000)
('Events Processed: ', 230000)
('Events Processed: ', 240000)
('Events Processed: ', 250000)
('Events Processed: ', 260000)
('Events Processed: ', 270000)
('Events Processed: ', 280000)
('Events Processed: ', 290000)
('Events Processed: ', 300000)
('Events Processed: ', 310000)
('Events Processed: ', 320000)
('Events Processe

('Events Processed: ', 500000)
(1.6823006249977202, 147.35953479247053, 3.846632315156566, 3.1590181678905083)
 FCN=1827.19 FROM MIGRAD    STATUS=CONVERGED     280 CALLS         281 TOTAL
                     EDM=7.55101e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.59035e+00   9.35569e-01   1.73307e-02   2.99695e-05
   2  p1           1.47346e+02   2.61046e+00   5.45277e-02   8.84217e-06
   3  p2           3.68354e+00   1.77969e+00   3.29587e-02  -5.78098e-05
   4  p3           3.24794e+00   6.49334e-02   1.34411e-03   2.84383e-04


In [13]:
# write this in for every bin i guess

bins = 1200
ROOT.enableJSVis()
c2 = ROOT.TCanvas("my2","Title",1000,1000)

diff = int(mean - 800)
print(diff)

h2 = Hist(bins,300,1500)

for i in range(bins):
    if (h1.GetBinContent(i) - offset >= 0):
        for j in range(int(h1.GetBinContent(i) - offset)):
            h2.Fill(i+300)
    #print(i+300,int(i-diff)+300,h1.GetBinContent(i))
    h2.SetBinContent(i-diff,h2.GetBinContent(i))

    
h2.Draw()
myFit.SetParameter(1,800)
myFit.SetParameter(2,10)
offset2 = h2.Fit('myFit','SQ').Get().Parameter(3)


#h3 = Hist(bins,300,1500)

#for i in range(bins):
#    if (h2.GetBinContent(i) - offset >= 0):
#        for j in range(int(h2.GetBinContent(i) - offset)):
#            h3.Fill(i+300)


#h3.SetTitle("Channel 1")
#c2.cd(2)
#h3.Draw()
#myFit.SetParameter(1,880)
#myFit.SetParameter(2,5)
#h3.Fit('myFit','','',850,900)
#offset3 = h3.Fit('myFit','SQ').Get().Parameter(3)
    
c2.Draw()



79
